In [4]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import csv

In [2]:
client = pymongo.MongoClient("127.0.0.1", 27017)

In [3]:
root_path = os.path.expanduser("/Users/dabao/Desktop/trading-data.20180427")

In [97]:
code = "600000"
file_name = "%s/stock data/sh%s.csv" % (root_path, code)
df = pandas.read_csv(file_name)

In [112]:
df = pandas.read_csv(file_name)
last_close = df["close"][1:].tolist() + [0]
df.insert(6, "last_close", last_close)
df["amount"] = df["money"]
df["tradeValue"] = df["traded_market_value"] / 10000
df["date"] = df["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
df["volume"] = df["volume"] / 100
df["change"] = df["change"] * 100

columns = ["date", "open", "close", "high", "low", "last_close",
           "change", "volume", "amount", "tradeValue"] 

data = df[columns].T.to_dict().values()

count = 0
collection = client.Trade[code]
for item in data:
    res = collection.replace_one({"date": item["date"]}, item)
    print(res, item)
    count = count + 1
    if count > 0:
        break
    

(<pymongo.results.UpdateResult object at 0x10c8afef0>, {'volume': 234507.83, 'tradeValue': 32628469.885577995, 'high': 11.67, 'amount': 272328553.0, 'last_close': 11.57, 'low': 11.55, 'date': Timestamp('2018-04-27 00:00:00'), 'close': 11.61, 'open': 11.58, 'change': 0.3457})


In [ ]:
df["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))